# Local Function Review

In [3]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
        
    return inner_function()

In [4]:
# Test it
outer_function()

Hi


In [5]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
    # Return the function. No ()    
    return inner_function

In [6]:
my_func = outer_function()
my_func

<function __main__.outer_function.<locals>.inner_function>

In [7]:
# Run it
my_func()

Hi


In [8]:
my_func()

Hi


In [12]:
# Now add some variation
def outer_function(msg):
    message = msg
    def inner_function():
        print(message)
        
    return inner_function

In [13]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

# Run it
hi_func()
bye_func()

Hi
Bye


In [15]:
# Now add some variables
# Remove intermediate members
def outer_function(msg):
    def inner_function():
        print(msg)
        
    return inner_function

In [16]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

# Run it
hi_func()
bye_func()

Hi
Bye


## Decorators
- A function that takes another function as argument
- Add some functionality
- Returns another function
- All of this without altering the original function

In [17]:
def decorator_function(original_function):
    def wrapper_function():
        return original_function()
    # Return function
    return wrapper_function

def display():
    print("Display function ran")

In [19]:
# Test it
decorated_display = decorator_function(display)
decorated_display

<function __main__.decorator_function.<locals>.wrapper_function>

In [20]:
decorated_display()

Display function ran


In [21]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper executed this before {}".format(
              original_function.__name__))
        return original_function()
    # Return function
    return wrapper_function

def display():
    print("Display function ran")

In [22]:
# Test it
decorated_display = decorator_function(display)
decorated_display

<function __main__.decorator_function.<locals>.wrapper_function>

In [23]:
decorated_display()

Wrapper executed this before display
Display function ran


## Now decorate your functions
**@Decorator_Function_name**

In [24]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper executed this before {}".format(
              original_function.__name__))
        return original_function()
    # Return function
    return wrapper_function

@decorator_function
def display():
    print("Display function ran")

In [25]:
# Test it
display()

Wrapper executed this before display
Display function ran


This will not work if our original function takes arguments

In [28]:
@decorator_function
def display_info(name, age):
    print("display_info ran with arguments ({}, {})".format(name, age))

In [29]:
display_info("Mario", 21)

TypeError: wrapper_function() takes 0 positional arguments but 2 were given

We can fix this with **\*args** and **\*\*kwargs**

In [30]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print("Wrapper executed this before {}".format(
              original_function.__name__))
        return original_function(*args, **kwargs)
    # Return function
    return wrapper_function

@decorator_function
def display():
    print("Display function ran")
    
@decorator_function
def display_info(name, age):
    print("display_info ran with arguments ({}, {})".format(name, age))

In [31]:
# Test it
display()

Wrapper executed this before display
Display function ran


In [32]:
display_info("Mario", 21)

Wrapper executed this before display_info
display_info ran with arguments (Mario, 21)


### Example: escape unicode

In [33]:
def escape_unicode(f):
    def wrap(*args, **kwargs):
        x = f(*args, **kwargs)
        return ascii(x)
    return wrap

@escape_unicode
def mexico_city():
    # Alt + 130 for é
    return "México"

In [34]:
mexico_city()

"'M\\xe9xico'"

#### What can be a Decorator (callable object)?
- Class Objects, which can be callable with the **dunder call**
- Functions as decorators

### Classes as Decorators

In [42]:
class DecoratorClass(object):
    def __init__(self, original_function):
        self.original_function = original_function
        
    def __call__(self, *args, **kwargs):
        print("Call method executed this before {}".format(
              self.original_function.__name__))
        return self.original_function(*args, **kwargs)
    
@DecoratorClass
def display():
    print("Display function ran")
    
@DecoratorClass
def display_info(name, age):
    print("display_info ran with arguments ({}, {})".format(name, age))

In [43]:
# Test it
display()
display_info("Mario", 21)

Call method executed this before display
Display function ran
Call method executed this before display_info
display_info ran with arguments (Mario, 21)


## Instances as Decorators
A Class instance

In [46]:
class Trace:
    def __init__(self):
        self._enable = True
        
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self._enable:
                print("Calling {}".format(f.__name__))
            return f(*args, **kwargs)
        return wrap
    
# Create an instance
tracer = Trace()
# Instance as decorator
@tracer
def rotate_list(l):
    return l[1:] + [l[0]]


Unlike our previous example, the **class object itself is not the decorator.** Rather, instances of Trace can be used as decorators.

In [47]:
# Test it
l1 = [1, 2, 3]
l1 = rotate_list(l1)

Calling rotate_list


In [48]:
l1

[2, 3, 1]

In [49]:
# Turn off the enable key from the instance
tracer._enable = False
l1 = [1, 2, 3]
l1 = rotate_list(l1)

## Back to function decorators

In [50]:
import time
def my_timer(original_function):
    """
    Helps you keep track of the loggin part of a function
    """
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(original_function.__name__, t2))
        return result
    
    return wrapper

# Test it
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran wwith arguments ({}, {})".format(
          name, age))

In [51]:
# Test it
display_info("Mario", 21)

Display_info ran wwith arguments (Mario, 21)
display_info ran in: 1.0020768642425537


## Multiple decorators

In [53]:
def my_logger(original_func):
    import logging
    logging.basicConfig(filename= '{}.log'.format(
                        original_func.__name__), 
                        level=logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info("Ran with args: {}, and kwargs: {}".format(
                    args, kwargs))
        return original_func(*args, **kwargs)
    return wrapper

@my_logger
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran wwith arguments ({}, {})".format(
          name, age))

In [54]:
# Test it
display_info("Weber", 125)

Display_info ran wwith arguments (Weber, 125)


In [55]:
# Test it with multiple decorators
@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({}, {})".format(
          name, age))
# Test it
display_info("Mario", 21)

Display_info ran wwith arguments (Mario, 21)
wrapper ran in: 1.0141632556915283


In [56]:
# Switch the order of the decorators
# Test it with multiple decorators
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({}, {})".format(
          name, age))
# Test it
display_info("Mario", 21)

Display_info ran wwith arguments (Mario, 21)
display_info ran in: 1.0011084079742432


Wrap ***everything!!!***

In [57]:
import time
from functools import wraps

def my_logger(original_func):
    import logging
    logging.basicConfig(filename= '{}.log'.format(
                        original_func.__name__), 
                        level=logging.INFO)
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        logging.info("Ran with args: {}, and kwargs: {}".format(
                    args, kwargs))
        return original_func(*args, **kwargs)
    
    return wrapper

def my_timer(original_function):
    @wraps(original_function)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(original_function.__name__, t2))
        return result
    
    return wrapper

In [58]:
# Test it with multiple decorators
@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({}, {})".format(
          name, age))
# Test it
display_info("Mario", 21)

Display_info ran with arguments (Mario, 21)
display_info ran in: 1.0023612976074219


In [59]:
# Switch the order of the decorators
# Test it with multiple decorators
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display_info ran with arguments ({}, {})".format(
          name, age))
# Test it
display_info("Mario", 21)

Display_info ran with arguments (Mario, 21)
display_info ran in: 1.000105857849121
